In [ ]:
import pandas as pd
import numpy as np
train_df = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv")
test_df = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv")

In [ ]:
train_df.head(5)

In [ ]:
import pickle

def get_cpc_info(cpc_code,summary=0):
    if summary ==  1 :
        txt = cpc_titles[cpc_code[0:1]] 
    else:
        if summary == 0:
            txt = cpc_titles[cpc_code]
        else:
            txt = ""
            for k,v in cpc_titles.items():
                if k.startswith(cpc_code):
                    #print(k)
                    txt += cpc_titles[k] + "."
    return txt.lower()

with open('/kaggle/input/cpc-titles-pickle/cpc_titles.pickle','rb') as handle:
    cpc_titles = pickle.load(handle)

In [ ]:
train_df['context_text']=train_df['context'].apply(lambda x: get_cpc_info(x))
test_df['context_text']=test_df['context'].apply(lambda x: get_cpc_info(x))

In [ ]:
train_df.head()

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
lemmatizer = WordNetLemmatizer()

def lemma_stopwords(sent):
    new_sent=[]
    filtered_words = [word for word in sent.split(" ") if word not in stopwords.words('english')]
    for x in filtered_words:
        new_sent.append(lemmatizer.lemmatize(x, pos ="v"))
    return " ".join(new_sent) 

In [ ]:
!pip install sentence-transformers

In [ ]:
train_df['text_a'] = (train_df['anchor']+'[SEP]'+train_df['context_text']).apply(lambda x: lemma_stopwords(x))
train_df['text_b'] = train_df['target'].apply(lambda x: lemma_stopwords(x))

sentence_pairs = train_df[["text_a", "text_b"]].values.astype("str")
y = train_df['score'].values.astype(np.float64)

In [ ]:
from sentence_transformers import SentenceTransformer 
strans = SentenceTransformer('AI-Growth-Lab/PatentSBERTa')
X = strans.encode(sentence_pairs)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
import xgboost as xgb
xgb.set_config(verbosity=2)
model = xgb.XGBRegressor()

In [ ]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
preds = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, preds)

In [ ]:
y_test

In [ ]:
preds

In [ ]:
test_df['text_a'] = (test_df['anchor']+'[SEP]'+test_df['context_text']).apply(lambda x: lemma_stopwords(x))
test_df['text_b'] = test_df['target'].apply(lambda x: lemma_stopwords(x))

sentence_pairs = test_df[["text_a", "text_b"]].values.astype("str")

X = strans.encode(sentence_pairs)
preds = model.predict(X)

In [ ]:
output_df = pd.DataFrame({'id': test_df['id'], 'score': pd.Series(preds)})
output_df.to_csv("submission.csv", index=False)